In [21]:
from sqlalchemy import create_engine
import pandas as pd

# Connecting to MySQL
en = create_engine("mysql+pymysql://root@localhost:3306/walmart_db")

In [22]:
pd.read_sql("SELECT * FROM walmart;", en)

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48,604.17
...,...,...,...,...,...,...,...,...,...,...,...,...
9964,9996,WALM056,Rowlett,Fashion accessories,37.00,3.0,03/08/23,10:10:00,Cash,3.0,0.33,111.00
9965,9997,WALM030,Richardson,Home and lifestyle,58.00,2.0,22/02/21,14:20:00,Cash,7.0,0.48,116.00
9966,9998,WALM050,Victoria,Fashion accessories,52.00,3.0,15/06/23,16:00:00,Credit card,4.0,0.48,156.00
9967,9999,WALM032,Tyler,Home and lifestyle,79.00,2.0,25/02/21,12:25:00,Cash,7.0,0.48,158.00


In [ ]:
# Number of distinct payment methods
pd.read_sql("SELECT DISTINCT(payment_method) FROM walmart", en)

,payment_method
0,Ewallet
1,Cash
2,Credit card


In [ ]:
# number of transactions from each payment method
pd.read_sql("""SELECT payment_method ,COUNT(*)
            FROM walmart 
            GROUP BY payment_method""",en)

,payment_method,COUNT(*)
0,Cash,1832
1,Credit card,4256
2,Ewallet,3881


In [ ]:
# Number of distinct brachs
pd.read_sql("""SELECT COUNT(DISTINCT(branch))
            FROM walmart""", en)

,COUNT(DISTINCT(branch))
0,100


In [28]:
# Maximumn quantity
pd.read_sql("""SELECT MAX(quantity)
            FROM walmart""",en)

,MAX(quantity)
0,10.0


In [29]:
# Minimum quantity
pd.read_sql("""SELECT MIN(quantity)
            FROM walmart""",en)

,MIN(quantity)
0,1.0


In [33]:
# Business Problem Q1: Find different payment methods, number of transactions, and quantity sold by payment method
pd.read_sql("""SELECT payment_method, COUNT(*) AS no_of_payments, SUM(quantity) AS quantity_sol
            FROM walmart
            GROUP BY payment_method""",en)

,payment_method,no_of_payments,quantity_sol
0,Cash,1832,4984.0
1,Credit card,4256,9567.0
2,Ewallet,3881,8932.0


In [47]:
# Project Question #2: Identify the highest-rated category in each branch
# Display the branch, category, and avg rating
pd.read_sql("""SELECT * FROM (SELECT branch, category, AVG(rating) AS avg_rating,
            RANK() OVER(PARTITION BY branch ORDER BY AVG(rating) DESC) AS rnk
            FROM walmart
            GROUP BY branch,category) T
            WHERE rnk=1""",en)

,branch,category,avg_rating,rnk
0,WALM001,Electronic accessories,7.450000,1
1,WALM002,Food and beverages,8.250000,1
2,WALM003,Sports and travel,7.500000,1
3,WALM004,Food and beverages,9.300000,1
4,WALM005,Health and beauty,8.366667,1
...,...,...,...,...
96,WALM096,Sports and travel,9.600000,1
97,WALM097,Food and beverages,7.675000,1
98,WALM098,Health and beauty,9.800000,1
99,WALM099,Electronic accessories,5.950000,1


In [ ]:
# Q3: Identify the busiest day for each branch based on the number of transactions
pd.read_sql("""SELECT * FROM 
            (SELECT branch, DAYNAME(date) AS day_name, COUNT(*) AS no_of_transactions,
            RANK() OVER(PARTITION BY branch ORDER BY COUNT(*) DESC) AS rnk
            FROM walmart
            GROUP BY branch,day_name) T
            WHERE rnk=1""",en)

,branch,day_name,no_of_transactions,rnk
0,WALM001,Sunday,18,1
1,WALM002,Saturday,16,1
2,WALM003,Sunday,30,1
3,WALM004,Sunday,16,1
4,WALM005,Thursday,15,1
...,...,...,...,...
117,WALM096,Tuesday,14,1
118,WALM097,Wednesday,15,1
119,WALM098,Wednesday,15,1
120,WALM099,Sunday,34,1


In [82]:
# Q4: Calculate the total quantity of items sold per payment method
pd.read_sql("""SELECT payment_method, SUM(quantity) AS qty_sold
            FROM walmart
            GROUP BY payment_method""",en)

,payment_method,qty_sold
0,Cash,4984.0
1,Credit card,9567.0
2,Ewallet,8932.0


In [86]:
# Q5: Determine the average, minimum, and maximum rating of categories for each city
pd.read_sql("""SELECT city, category, MAX(rating), MIN(rating), AVG(rating)
            FROM walmart
            GROUP BY city, category""",en)

,city,category,MAX(rating),MIN(rating),AVG(rating)
0,Abilene,Electronic accessories,8.8,7.1,7.966667
1,Abilene,Fashion accessories,9.0,4.0,6.240625
2,Abilene,Food and beverages,8.9,6.0,6.950000
3,Abilene,Health and beauty,9.7,9.7,9.700000
4,Abilene,Home and lifestyle,9.0,4.0,6.096875
...,...,...,...,...,...
508,Weslaco,Fashion accessories,9.5,3.0,5.042012
509,Weslaco,Food and beverages,9.8,7.1,8.733333
510,Weslaco,Health and beauty,9.2,4.3,6.750000
511,Weslaco,Home and lifestyle,9.2,3.0,5.183505


In [91]:
# Q6: Calculate the total profit for each category
pd.read_sql("""SELECT category, SUM(total * profit_margin) AS total_profit
            FROM walmart
            GROUP BY category""",en)

,category,total_profit
0,Electronic accessories,30772.4895
1,Fashion accessories,192314.8932
2,Food and beverages,21552.8622
3,Health and beauty,18671.7345
4,Home and lifestyle,192213.6381
5,Sports and travel,20613.8082


In [98]:
# Q7: Determine the most common payment method for each branch
pd.read_sql("""SELECT * FROM (
            SELECT branch, payment_method, COUNT(*) AS no_of_transactions,
            RANK() OVER(PARTITION BY branch ORDER BY COUNT(*) DESC) AS rnk
            FROM walmart
            GROUP BY branch, payment_method) T
            WHERE rnk=1""",en)

,branch,payment_method,no_of_transactions,rnk
0,WALM001,Ewallet,45,1
1,WALM002,Ewallet,37,1
2,WALM003,Credit card,115,1
3,WALM004,Ewallet,44,1
4,WALM005,Ewallet,56,1
...,...,...,...,...
95,WALM096,Ewallet,50,1
96,WALM097,Ewallet,38,1
97,WALM098,Ewallet,44,1
98,WALM099,Credit card,83,1


In [112]:
# Q8: Categorize sales into Morning, Afternoon, and Evening shifts
# Find no of invoices in each shift and branch
pd.read_sql("""
    SELECT branch,
        CASE 
            WHEN HOUR(time) < 12 THEN 'Morning'
            WHEN HOUR(time) BETWEEN 12 AND 17 THEN 'Afternoon'
            ELSE 'Evening'
        END AS shift,
        COUNT(*) AS no_of_invoices
    FROM walmart
    GROUP BY branch, shift
    ORDER BY branch, COUNT(*) DESC;
""", en)


,branch,shift,no_of_invoices
0,WALM001,Afternoon,36
1,WALM001,Evening,30
2,WALM001,Morning,8
3,WALM002,Afternoon,29
4,WALM002,Evening,21
...,...,...,...
295,WALM099,Morning,40
296,WALM099,Evening,32
297,WALM100,Afternoon,24
298,WALM100,Morning,19


In [138]:
# Q9: Identify the 5 branches with the highest revenue decrease ratio from 2022 to 2023
pd.read_sql("""SELECT *, ROUND(((rev_2022-rev_2023)/rev_2022)*100, 2) AS ratio FROM 
            ( SELECT t1.branch, rev_2023, rev_2022 FROM 
            (SELECT branch, SUM(total) AS rev_2023
            FROM walmart
            WHERE YEAR(date)=2023
            GROUP BY branch) t1
            JOIN
            (SELECT branch, SUM(total) AS rev_2022
            FROM walmart
            WHERE YEAR(date)=2022
            GROUP BY branch) t2
            ON t1.branch=t2.branch) t
            ORDER BY ratio DESC
            LIMIT 5""",en)

,branch,rev_2023,rev_2022,ratio
0,WALM084,48.0,755.00,93.64
1,WALM075,105.0,1416.64,92.59
2,WALM024,60.0,629.00,90.46
3,WALM013,16.0,114.00,85.96
4,WALM060,81.0,470.86,82.80
